In [1]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytesseract
from PIL import Image
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

pytesseract.pytesseract.tesseract_cmd="C:/Program Files/Tesseract-OCR/tesseract.exe"

In [2]:
img_path = "F:/Envirya_Project/Images/Capture.png"
file_name = os.path.splitext(img_path)[0]
file_name = file_name.split("/")[-1]

img = cv2.imread(img_path,0) #Grayscale Image
img = cv2.GaussianBlur(img, (3,3), 0)
thresh, img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY |cv2.THRESH_OTSU)
#Inversion of image
img_bin = 255-img_bin
# cv2.imwrite("trial_thresh.png",img_bin)
# cv2.imshow("Gray",img_bin)
# cv2.waitKey()
# cv2.destroyAllWindows()

np.array(img).shape
img.shape #height,width

#start the kernel part
length = img.shape[1]//100 #width
kernel_ver = cv2.getStructuringElement(cv2.MORPH_RECT,(1,length)) 
kernel_hor = cv2.getStructuringElement(cv2.MORPH_RECT,(length,1))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))

<IPython.core.display.Javascript object>

(452, 1207)

(452, 1207)

In [3]:
#Detection of vertical lines
img_ver = cv2.erode(img_bin,kernel_ver,iterations=3)
lines_ver = cv2.dilate(img_ver,kernel_ver,iterations=3)
# cv2.imshow("Vertical",lines_ver)

#Detection of horizontal lines
img_hor = cv2.erode(img_bin,kernel_hor,iterations=3)
lines_hor = cv2.dilate(img_hor,kernel_hor,iterations=3)
# cv2.imshow("Horizontal",lines_hor)

#Combining both the lines to a new image.
comb_img = cv2.addWeighted(lines_ver,0.5,lines_hor,0.5,0.0)
comb_img = cv2.erode(~comb_img, kernel, iterations=2)
thresh, comb_img = cv2.threshold(comb_img, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# cv2.imshow("Combined",comb_img)

bitxor = cv2.bitwise_xor(img,comb_img)
bitnot = cv2.bitwise_not(bitxor)
# cv2.imshow("BIT Operated",bitnot)

contours, hierarchy = cv2.findContours(comb_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#Sorting the Contours
def sort_contour(conts, method):
    i = 0
    rev_order = False
    if method == "right-to-left" or method == "bottom-to-top":
        rev_order = True
    if method == "top-to-bottom" or method == "left-to-right":
        i = 1
    bounding_boxes = [cv2.boundingRect(contr) for contr in conts]
    (conts,bounding_boxes) = zip(*sorted(zip(conts,bounding_boxes),key=lambda x:x[1][i], reverse=rev_order))
    return (conts, bounding_boxes)

contours, bounding_boxes = sort_contour(contours, method="top-to-bottom")
# print(bounding_boxes)
# print(bounding_boxes[1][3])

#saving the heights of all detected boxes
heights = [bounding_boxes[i][3] for i in range(len(bounding_boxes))]
mean_height = np.mean(heights)

cells=[]
#get the width,height of each cell, and display the contour
for cntr in contours:
    x, y, w, h = cv2.boundingRect(cntr)
    if (w<1000 and h<500):
        image= cv2.rectangle(img, (x,y), (x+w, y+h),(0,255,0),2)
        cells.append([x, y, w, h])
    
# cv2.imshow("Highlighted",image)
# print(f"Cells {cells}")

# cv2.waitKey()
# cv2.destroyAllWindows()

In [4]:
#Differentiating rows and columns
row = []
col = []
j = 0

for idx in range(len(cells)):
    if idx == 0:
        col.append(cells[idx])
        temp = cells[idx]
    else:
        if(cells[idx][1] <= temp[1]+mean_height/2):
            col.append(cells[idx])
            temp = cells[idx]
            
            if(idx==len(cells)-1):
                row.append(col)
        else:
            row.append(col)
            col = []
            temp = cells[idx]
            col.append(cells[idx])
            
# Understand the the columns required in final dataframe
max_col = 0
count = 0
for i in range(len(row)):
    count = len(row[i])
    if count > max_col:
        max_col = count
        
# print(row)
# print(f"Max Cols: {max_col}")
#get the column centers to align properly
center = [int(row[i][j][0]+row[i][j][2]/2) for j in range(len(row[i])) if row[0]]
center=np.array(center)
center.sort()
    
final_cells = []
for i in range(len(row)):
    temp_cell = []
    for j in range(max_col):
        temp_cell.append([])
    for k in range(len(row[i])):
        diff = abs(center-(row[i][k][0]+row[i][k][2]/4))
        minimum = min(diff)
        indexing = list(diff).index(minimum)
        temp_cell[indexing].append(row[i][k])
    final_cells.append(temp_cell)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

Rows
[[[981, 100, 155, 22], [820, 100, 156, 22], [661, 100, 155, 22], [501, 100, 156, 22], [341, 100, 156, 22], [181, 100, 155, 22], [21, 100, 156, 22], [829, 106, 78, 10], [349, 106, 58, 6]], [[981, 126, 155, 21], [820, 126, 156, 21], [661, 126, 155, 21], [501, 126, 156, 21], [341, 126, 156, 21], [181, 126, 155, 21], [21, 126, 156, 21], [829, 132, 78, 10]], [[981, 157, 155, 23], [820, 157, 156, 23], [661, 157, 155, 23], [501, 157, 156, 23], [181, 157, 316, 23], [21, 157, 156, 23], [724, 163, 44, 5], [70, 163, 45, 5], [870, 164, 43, 5], [314, 164, 58, 5], [79, 168, 51, 5]], [[981, 184, 155, 22], [820, 184, 156, 22], [661, 184, 155, 22], [501, 184, 156, 22], [181, 184, 316, 22], [21, 184, 156, 22]], [[981, 210, 155, 22], [820, 210, 156, 22], [661, 210, 155, 22], [501, 210, 156, 22], [181, 210, 316, 22], [21, 210, 156, 22]], [[981, 237, 155, 20], [820, 237, 156, 20], [661, 237, 155, 20], [501, 237, 156, 20], [181, 237, 316, 20], [21, 237, 156, 20]], [[981, 262, 155, 22], [820, 262, 156, 

-1

In [5]:
#extraction of data from each cell using pytesseract
data = []
for i in range(len(final_cells)):
    for j in range(len(final_cells[i])):
        temp_data = ""
        if len(final_cells[i][j])==0:
            data.append(" ")
        else:
            for k in range(len(final_cells[i][j])):
                y,x,w,h = final_cells[i][j][k][0], final_cells[i][j][k][1], final_cells[i][j][k][2], final_cells[i][j][k][3]
                final_img = bitnot[x:x+h, y:y+w]
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
                border = cv2.copyMakeBorder(final_img,2,2,2,2,   cv2.BORDER_CONSTANT,value=[255,255])
                resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                dilation = cv2.dilate(resizing, kernel,iterations=1)
                erosion = cv2.erode(dilation, kernel,iterations=1)
                text = pytesseract.image_to_string(erosion)
                if len(text) == 0:
                    text = pytesseract.image_to_string(erosion, config='--psm 3')
                temp_data += " "+text
            data.append(temp_data)
            
#creating a dataframe from the list and then saving as csv file
data_arr = np.array(data)
df = pd.DataFrame(data_arr.reshape(len(row),max_col))
df.to_csv("F:/Envirya_Project/csv_files/" + file_name + ".csv")
final = df.style.set_properties(align="left")
# print(type(final))
final.to_excel("F:/Envirya_Project/csv_files/" + file_name + ".xlsx")